# Laboratorio 9

In [63]:
from zipfile import ZipFile
from pathlib import Path
import pandas as pd

Path.cwd()

WindowsPath('d:/Documenti/Git_repositories/Data_science_lab/data-science-lab-1/MyRepo/Lab9')

In [64]:
with ZipFile('NYC_Airbnb.zip') as zipped_file:
    zipped_file.extractall(Path.cwd())

1. Load the dataset from the root folder

In [65]:
df_dev = pd.read_csv(Path.cwd()/'NYC_Airbnb'/'development.csv', index_col=0)
df_eval = pd.read_csv(Path.cwd()/'NYC_Airbnb'/'evaluation.csv', index_col=0)

In [66]:
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39116 entries, 12783632 to 15310580
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            39103 non-null  object 
 1   host_id                         39116 non-null  int64  
 2   host_name                       39097 non-null  object 
 3   neighbourhood_group             39116 non-null  object 
 4   neighbourhood                   39116 non-null  object 
 5   latitude                        39116 non-null  float64
 6   longitude                       39116 non-null  float64
 7   room_type                       39116 non-null  object 
 8   price                           39116 non-null  int64  
 9   minimum_nights                  39116 non-null  int64  
 10  number_of_reviews               39116 non-null  int64  
 11  last_review                     31075 non-null  object 
 12  reviews_per_month     

In [67]:
df_dev.head()

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,,,,,,
12783632,NYC Mini Hotel,57230304,Imanuelly,Queens,Elmhurst,40.74037,-73.88610,Private room,75,1,2,2019-05-26,0.92,3,351
3463385,Gorgeous room in Manhattan,10698270,Evgenia,Manhattan,Upper East Side,40.76717,-73.95532,Private room,95,1,202,2019-05-27,3.31,2,263
17572926,Great 1 Bedroom on Upper East,36578169,James,Manhattan,Upper East Side,40.77984,-73.94725,Entire home/apt,130,2,0,NaN,NaN,1,0
33913644,"Modern and bright 2Bed 2Bath Bushwick, Brooklyn",50981314,Ofier,Brooklyn,Bushwick,40.70205,-73.91338,Entire home/apt,150,2,4,2019-07-07,1.64,1,89
9405895,Stylish and zen Brooklyn retreat,48775347,Mathieu,Brooklyn,Fort Greene,40.68914,-73.97853,Entire home/apt,325,3,16,2019-04-20,0.42,1,103


In [68]:
df = pd.concat([df_dev,df_eval],axis=0)

In [69]:
df.isna().any()

name                               True
host_id                           False
host_name                          True
neighbourhood_group               False
neighbourhood                     False
latitude                          False
longitude                         False
room_type                         False
price                              True
minimum_nights                    False
number_of_reviews                 False
last_review                        True
reviews_per_month                  True
calculated_host_listings_count    False
availability_365                  False
dtype: bool

In [70]:
df['reviews_per_month'].fillna(0,inplace=True)

Dummy:  
'neighbourhood_group', 'neighbourhood','room_type'  
Discard:  
'host_id','name','host_name','last_review'

In [71]:
df_1h = pd.get_dummies(df,columns=['neighbourhood_group', 'neighbourhood','room_type'])
df_dropped = df_1h.drop(['host_id','name','host_name','last_review'], axis=1)

In [91]:
from sklearn.model_selection import train_test_split

train_and_val_mask = df_dropped['price'].notna()

feature_names = df_dropped.drop('price',axis=1).columns

X = df_dropped.loc[:,list(feature_names)].values
y = df_dropped['price'].values

X_train_and_val = X[train_and_val_mask]
y_train_and_val = y[train_and_val_mask]
X_test = X[~train_and_val_mask]
y_test = y[~train_and_val_mask]

X_train, X_valid, y_train, y_valid= train_test_split(X_train_and_val, y_train_and_val, shuffle=True, random_state=42)

In [94]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-2)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_valid)
r2_score(y_valid,y_pred)

0.07128877883880957

In [104]:
reg_feature_importances_1 = pd.Series(reg.feature_importances_, index=feature_names, name='importances').sort_values(ascending=False)
reg_feature_importances_1

longitude                       2.011533e-01
latitude                        2.007119e-01
availability_365                9.798847e-02
minimum_nights                  8.811410e-02
reviews_per_month               6.634125e-02
                                    ...     
neighbourhood_New Brighton      1.698312e-09
neighbourhood_Richmondtown      0.000000e+00
neighbourhood_Neponsit          0.000000e+00
neighbourhood_Woodrow           0.000000e+00
neighbourhood_Fort Wadsworth    0.000000e+00
Name: importances, Length: 236, dtype: float64

Dummy:  
'neighbourhood_group', 'neighbourhood','room_type'  
Discard:  
'host_id','name','host_name','last_review'

In [114]:
df_1h = pd.get_dummies(df, columns=['room_type'])
df_dropped = df_1h.drop(['neighbourhood_group', 'neighbourhood','host_id','name','host_name','last_review'],axis=1)

train_and_val_mask = df_dropped['price'].notna()

feature_names = df_dropped.drop('price',axis=1).columns

X = df_dropped.drop('price',axis=1).values
y = df_dropped['price'].values

X_train_and_val = X[train_and_val_mask]
y_train_and_val = y[train_and_val_mask]
X_test = X[~train_and_val_mask]
y_test = y[~train_and_val_mask]

X_train, X_valid, y_train, y_valid= train_test_split(X_train_and_val, y_train_and_val, shuffle=True, random_state=42)

In [115]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-2)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_valid)
r2_score(y_valid,y_pred)

0.10485905409912633

In [116]:
reg_feature_importances_1 = pd.Series(reg.feature_importances_, index=feature_names, name='importances').sort_values(ascending=False)
reg_feature_importances_1

longitude                         0.310636
latitude                          0.243221
availability_365                  0.102062
minimum_nights                    0.094534
reviews_per_month                 0.070677
room_type_Entire home/apt         0.063075
calculated_host_listings_count    0.060979
number_of_reviews                 0.053690
room_type_Shared room             0.000705
room_type_Private room            0.000421
Name: importances, dtype: float64

Including 'id' and 'host_id' in the analysis

In [117]:
df_1h = pd.get_dummies(df, columns=['room_type'])
df_1h['id'] = df_1h.index
df_dropped = df_1h.drop(['neighbourhood_group', 'neighbourhood','name','host_name','last_review'],axis=1)

train_and_val_mask = df_dropped['price'].notna()

feature_names = df_dropped.drop('price',axis=1).columns

X = df_dropped.drop('price',axis=1).values
y = df_dropped['price'].values

X_train_and_val = X[train_and_val_mask]
y_train_and_val = y[train_and_val_mask]
X_test = X[~train_and_val_mask]
y_test = y[~train_and_val_mask]

X_train, X_valid, y_train, y_valid= train_test_split(X_train_and_val, y_train_and_val, shuffle=True, random_state=42)

In [119]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-2)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_valid)
r2_score(y_valid,y_pred)

0.13278619065870456

In [120]:
reg_feature_importances_1 = pd.Series(reg.feature_importances_, index=feature_names, name='importances').sort_values(ascending=False)
reg_feature_importances_1

longitude                         0.203147
host_id                           0.176144
id                                0.146732
latitude                          0.130673
minimum_nights                    0.078468
availability_365                  0.077420
room_type_Entire home/apt         0.063075
reviews_per_month                 0.043341
calculated_host_listings_count    0.043281
number_of_reviews                 0.037004
room_type_Private room            0.000435
room_type_Shared room             0.000280
Name: importances, dtype: float64

### Exploration of the post titles

In [123]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", binary=True, use_idf=False, norm=False)

wpm = vectorizer.fit_transform(df_1h["name"].fillna(""))

print(wpm[:5,:5])